# Step 1: Understanding the Data

In [ ]:

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
import datetime
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from datetime import datetime 

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline




In [ ]:
df1= pd.read_excel("./model_data1.xlsx")
df1 = df1.fillna(df1.mean())
df1

In [ ]:
df2= pd.read_excel("./model_data2.xlsx")

df2

In [ ]:
df3= pd.concat([df1[df1.columns[:-1]],df2],axis=1)
df3

In [ ]:
## 可视化变量相关性

col = ["front_PCA1", "Roof subsidence", "Surface elevation" , "Burial depth", "side_tSNE1", "side_PCA5", "front_PCA5","side_PCA2","front_tSNE3","Maximum velocity of shock wave","Roof tensile stress"]
# 计算相关性矩阵
df4  = df3[col]
correlation_matrix = df4.corr()
print(correlation_matrix )
plt.rcParams['font.family'] = 'Times New Roman'

# 绘制热力图
plt.figure(figsize=(12, 12))
ax = sns.heatmap(correlation_matrix, 
                 annot=False, fmt=".2f", cmap='RdBu_r',  # 设置annot=False, 不自动注释
                 linewidths=0.5, linecolor='white', 
                 cbar_kws={'shrink': 0.8})  # 缩小颜色条

# 手动添加相关系数数值
for i in range(correlation_matrix.shape[0]):
    for j in range(correlation_matrix.shape[1]):
        ax.text(j + 0.5, i + 0.5, f"{correlation_matrix.iloc[i, j]:.2f}",
                ha='center', va='center', 
                fontsize=12, fontweight='bold', color='black')

# 增大X、Y轴标签的字体大小并加粗
plt.xticks(fontsize=18, fontweight='bold')
plt.yticks(fontsize=18, fontweight='bold')

# 保存图像
plt.savefig('./correlation_matrix.png', dpi=800)
plt.show()


In [ ]:

describe_df = df3.describe().T
describe_df


In [ ]:
describe_df.to_csv('describe_df.csv')
print("describe_df.csv is saved in the current directory")

# Step 2: Model Training and Evaluation

## Step 2.1: Define Model Evaluation Metric Algorithms

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, make_scorer
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib
import sklearn
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
import xgboost as xgb
from sklearn.ensemble import GradientBoostingRegressor
import joblib
from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score, cross_validate, KFold, train_test_split
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from catboost import CatBoostRegressor
from ngboost import NGBRegressor
import lightgbm as lgb


def evaluate_regression_model(model, X_test, y_test):
    """
    Function to evaluate the performance of a regression model.

    Parameters:
        model: The trained model to evaluate.
        X_test (np.ndarray): Features of the test dataset.
        y_test (np.ndarray): True labels of the test dataset.

    Returns:
        dict: A dictionary containing performance metrics of the model.
    """

    y_pred = model.predict(X_test)


    metrics = {
        'R^2': r2_score(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'MBE': np.mean(y_test - y_pred),
        'AIC': len(y_test) * np.log(mean_squared_error(y_test, y_pred)) + 2 * X_test.shape[1],
        'BIC': len(y_test) * np.log(mean_squared_error(y_test, y_pred)) + np.log(len(y_test)) * X_test.shape[1],
        'SI': np.sqrt(mean_squared_error(y_test, y_pred)) / np.mean(y_test),
        'KGE': 1 - np.sqrt((r2_score(y_test, y_pred) - 1) ** 2 + (np.std(y_pred) / np.std(y_test) - 1) ** 2 + (np.mean(y_pred) / np.mean(y_test) - 1) ** 2)
    }

    return metrics




## Step 2.2: Split Training and Testing Data

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


def data_loader(df):
    X = df[df.columns[:-1]]
    y = df[df.columns[-1]]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = pd.DataFrame(X_scaled, columns=X.columns)
    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=12)

    return X_train, X_test, y_train, y_test 


##   Step 2.3: Perform Hyperparameter Optimization (Train the Model and Save the Best Model)

In [ ]:
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import numpy as np
import joblib
import xgboost as xgb


param_grid_knn = {'n_neighbors': [3, 5, 7, 9, 10, 15, 20]}
param_grid_xgb = {'max_depth': [2, 3, 5, 10, 15, 20, 50], 'learning_rate': [0.001, 0.01, 0.1,1], 'n_estimators': [10, 20, 30, 50, 100, 200, 300], 'subsample': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7]}
param_grid_rf = {'n_estimators': [10, 20, 30, 50, 100, 150, 200, 300, 400], 'max_depth': [3, 5, 7, 10, 15, 20, 50, 100], 'min_samples_split': [2, 4, 6, 8]}
param_grid_ann = {'hidden_layer_sizes': [(3,), (5,), (3, 3), (5, 5), (10,), (10, 10), (20,), (20, 20), (50,), (100,), (50, 50), (100, 100)], 'activation': ['tanh', 'relu'], 'alpha': [0.0001, 0.001, 0.01, 0.1, 10], 'max_iter': [200, 500, 1000]}
param_grid_lasso = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10]}
param_grid_elasticnet = {'alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]}
param_grid_adaboost = {'n_estimators': [50, 100, 200, 300], 'learning_rate': [0.001, 0.01, 0.1, 0.5, 1]}



svr_model = SVR()
knn_model = KNeighborsRegressor()
xgb_model = xgb.XGBRegressor(use_label_encoder=False, eval_metric='rmse')
rf_model = RandomForestRegressor()
ann_model = MLPRegressor()
lasso_model = Lasso()
elasticnet_model = ElasticNet()
adaboost_model = AdaBoostRegressor()


models = {
    'Lasso': (lasso_model, param_grid_lasso),
    'KNN': (knn_model, param_grid_knn),
    'XGBoost': (xgb_model, param_grid_xgb),
    'RandomForest': (rf_model, param_grid_rf),
    'AdaBoost': (adaboost_model, param_grid_adaboost) ,
    'ANN': (ann_model, param_grid_ann),
    'ElasticNet': (elasticnet_model, param_grid_elasticnet)
    
}




In [ ]:

datalist = [df1,df2,df3]
for i in range(len(datalist)):
    X_train, X_test, y_train, y_test = data_loader(datalist[i])
    data_name = "df"+str(i+1)
    eval_results = []
    for model_name, (model, param_grid) in models.items():
        print(f"\n训练{data_name} - {model_name}...")
        if param_grid:
            grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
            grid_search.fit(X_train, y_train)
            best_model = grid_search.best_estimator_
            joblib.dump(best_model, f'./{data_name}_{model_name}.pkl')
            print(f"{model_name} best_params ：{grid_search.best_params_}")
        else:
            model.fit(X_train, y_train)
            best_model = model
            joblib.dump(best_model, f'./{model_name}.pkl')
        

        metrics = evaluate_regression_model(best_model, X_test, y_test)
        metrics['Model'] = model_name
        eval_results.append(metrics)
        print(f"{model_name} evaluation result：")
        for metric, value in metrics.items():
            print(f"{metric}: {value}")

    eval_result_df = pd.DataFrame(eval_results)
    eval_result_df.to_excel(f'./{data_name}_model_evaluation_results.xlsx', index=False)

    print(f"{data_name}all model evaluation results saved in {data_name}_model_evaluation_results.xlsx。")

print("train is ok")

In [ ]:
eval_result_df1 = pd.read_excel("./df1_model_evaluation_results.xlsx")
eval_result_df1 

In [ ]:
eval_result_df2 = pd.read_excel("./df2_model_evaluation_results.xlsx")
eval_result_df2

In [ ]:
eval_result_df3 = pd.read_excel("./df3_model_evaluation_results.xlsx")
eval_result_df3

# Step 3: Validate Data Input Effectiveness (Using Non-Parametric Testing)



## **T-Test**
The T-test is used to determine whether there is a significant difference between the means of two groups. It is particularly suitable when:
- The data follows a **normal distribution**.
- The variances between the two groups are **homogeneous** (similar).

### Key Applications:
- Comparing model performance on different datasets.
- Assessing whether feature engineering steps improve prediction accuracy.

---

## **Wilcoxon Test**
The Wilcoxon test is a **non-parametric test** that is ideal for scenarios where:
- The data **does not follow a normal distribution**.
- The sample size is **small**.

### Key Characteristics:
- It compares the **medians** of two groups to assess significant differences.
- Useful for evaluating metrics where the distribution is unknown or deviates from normality.

### Common Use Cases:
- Testing input feature effectiveness when data distributions are skewed.
- Assessing robustness of model predictions under varying input conditions. 

By applying these tests appropriately, we can ensure rigorous validation of model performance and input feature reliability.

##  Step 3.1: Perform Validation on Basic Features and Image Features

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind, wilcoxon, shapiro

metrics = ["R^2", "RMSE", "MAE", "MBE", "AIC", "BIC", "SI","KGE"]
results = pd.DataFrame(index=metrics, columns=["T-Test p-value", "Wilcoxon p-value", "Normality p-value (df1)", "Normality p-value (df2)"])


for metric in metrics:

    norm_p_val_df1 = shapiro(eval_result_df1[metric])[1]
    norm_p_val_df2 = shapiro(eval_result_df2[metric])[1]


    if norm_p_val_df1 > 0.05 and norm_p_val_df2 > 0.05: 

        t_stat, p_val_t = ttest_ind(eval_result_df1[metric], eval_result_df2[metric])
        results.loc[metric, "T-Test p-value"] = p_val_t
    else:

        w_stat, p_val_w = wilcoxon(eval_result_df1[metric], eval_result_df2[metric])
        results.loc[metric, "Wilcoxon p-value"] = p_val_w

    results.loc[metric, "Normality p-value (df1)"] = norm_p_val_df1
    results.loc[metric, "Normality p-value (df2)"] = norm_p_val_df2

results


## Step 3.2: Perform Validation on Basic Features and Merged Features

In [ ]:
for metric in metrics:

    norm_p_val_df1 = shapiro(eval_result_df1[metric])[1]
    norm_p_val_df2 = shapiro(eval_result_df3[metric])[1]


    if norm_p_val_df1 > 0.05 and norm_p_val_df2 > 0.05:

        t_stat, p_val_t = ttest_ind(eval_result_df1[metric], eval_result_df3[metric])
        results.loc[metric, "T-Test p-value"] = p_val_t
    else:

        w_stat, p_val_w = wilcoxon(eval_result_df1[metric], eval_result_df2[metric])
        results.loc[metric, "Wilcoxon p-value"] = p_val_w

    results.loc[metric, "Normality p-value (df1)"] = norm_p_val_df1
    results.loc[metric, "Normality p-value (df2)"] = norm_p_val_df2

results

## Step 3.3: Perform Validation on Image Features and Merged Features

In [ ]:
for metric in metrics:

    norm_p_val_df1 = shapiro(eval_result_df2[metric])[1]
    norm_p_val_df2 = shapiro(eval_result_df3[metric])[1]


    if norm_p_val_df1 > 0.05 and norm_p_val_df2 > 0.05:  

        t_stat, p_val_t = ttest_ind(eval_result_df2[metric], eval_result_df3[metric])
        results.loc[metric, "T-Test p-value"] = p_val_t
    else:

        w_stat, p_val_w = wilcoxon(eval_result_df1[metric], eval_result_df2[metric])
        results.loc[metric, "Wilcoxon p-value"] = p_val_w

    results.loc[metric, "Normality p-value (df1)"] = norm_p_val_df1
    results.loc[metric, "Normality p-value (df2)"] = norm_p_val_df2

results

# Step 4: Visualization of Prediction Results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))  
plt.hist(df3[df3.columns[-1]], bins=20, color='blue', edgecolor='black')  
plt.title('Histogram')  
plt.xlabel('Values')  
plt.ylabel('Frequency') 
plt.show()


In [ ]:

import matplotlib.pyplot as plt
from matplotlib.colors import BoundaryNorm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
import joblib
import seaborn as sns
from matplotlib.colors import Normalize


plt.rcParams['font.family'] = 'Times New Roman'
colors = ['black', 'red', 'yellow', 'green', 'cyan', 'blue']
bounds = [0,0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.8] 

def plt_results(model, model_name, X_test, y_test, fig_name):
    plt.rcParams['font.family'] = 'Times New Roman'
    """
    Plot a scatter plot of predicted vs. actual values.

    Parameters:
        model: The trained model.
        X_test (np.ndarray): Test features.
        y_test (np.ndarray or pandas.Series): True labels of the test dataset.
        fig_name (str): Filename to save the plot.
    """

    y_pred = model.predict(X_test)

    y_test = np.array(y_test)


    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mbe = np.mean(y_test - y_pred)
    nse = 1 - (sum((y_test - y_pred) ** 2) / sum((y_test - np.mean(y_test)) ** 2))
    kge = 1 - np.sqrt((r2_score(y_test, y_pred) - 1) ** 2 + (np.std(y_pred) / np.std(y_test) - 1) ** 2 + (np.mean(y_pred) / np.mean(y_test) - 1) ** 2)
    si = rmse / np.mean(y_test)

    plt.figure(figsize=(16, 16),dpi=800)
    norm = Normalize(vmin=min(bounds), vmax=max(bounds))
    scatter = plt.scatter(y_test, y_pred, c=y_test, cmap='jet', norm=norm)
    plt.colorbar(scatter, ticks=bounds, boundaries=bounds)
    
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], 'purple', linestyle='--', label='Predicted Y = Observed Y')

    reg = LinearRegression().fit(y_test.reshape(-1, 1), y_pred)
    y_lin_pred = reg.predict(y_test.reshape(-1, 1))
    plt.plot(y_test, y_lin_pred, color='red', linestyle='-', label='Fitted line')


    se = np.sqrt(np.sum((y_pred - y_lin_pred) ** 2) / (len(y_test) - 2))
    t_val = 1.96  # The 95% confidence interval will be added to visualize the model's uncertainty or prediction interval.
    ci = t_val * se * np.sqrt(1 / len(y_test) + (y_test - np.mean(y_test))**2 / np.sum((y_test - np.mean(y_test))**2))
    sorted_indices = np.argsort(y_test)
    plt.fill_between(y_test[sorted_indices], 
                    y_lin_pred[sorted_indices] - ci[sorted_indices], 
                    y_lin_pred[sorted_indices] + ci[sorted_indices], 
                    color='pink', alpha=0.5, label='95% Confidence Interval')

    # 显示线性拟合方程和R2
    plt.text(min(y_test), (max(y_test) - min(y_test)) / 2, f'Y={reg.intercept_:.4f}+{reg.coef_[0]:.4f}X\n$R^2$={r2_score(y_test, y_pred):.3f}', 
             fontsize=24, color='black', verticalalignment='center', horizontalalignment='left', bbox=dict(facecolor='white', alpha=0.8))

    # 显示评估结果
    plt.text(max(y_test), min(y_pred), f'WIA={1 - sum((y_test - y_pred) ** 2) / sum((np.abs(y_pred - np.mean(y_test)) + np.abs(y_test - np.mean(y_test))) ** 2):.3f}\n'
                                       f'MAE={mae:.3f}\n'
                                       f'MBE={mbe:.3f}\n'
                                       f'KGE={kge:.3f}\n'
                                       f'NSE={nse:.3f}\n'
                                       f'SI={si:.3f}\n'
                                       f'RMSE={rmse:.3f}',
             fontsize=15, color='black', horizontalalignment='right', verticalalignment='bottom')


    plt.xlabel('Observed values', fontsize=36)
    plt.ylabel(f'{model_name} Predicted values', fontsize=36)
    plt.xticks(fontsize=24)
    plt.yticks(fontsize=24)
    plt.legend(fontsize=20) 
    plt.grid(True)
    
    plt.savefig(fig_name)
    plt.show()


estimator_name = ['ElasticNet','Lasso', 'KNN', 'XGBoost', 'RandomForest','AdaBoost', 'ANN']


for model_name in estimator_name:
    filename = f'./df3_{model_name}.pkl'
    estimator = joblib.load(filename)  
    fig_name = f'./{model_name}_prediction.png'
    plt_results(estimator, model_name, X_test, y_test, fig_name)

# Step 5: Use SHAP to Interpret the Random Forest (RF) and Perform Knowledge Discovery

### **Detailed Steps**:

1. **Generate SHAP Values**:
   - Use the SHAP library to compute SHAP values for the Random Forest model.
   - SHAP values quantify the contribution of each feature to the model's predictions.

2. **Global Interpretation**:
   - **Summary Plot**: Visualize the average impact of each feature across all samples to identify the most influential features.
   - **Feature Importance**: Aggregate SHAP values to rank features based on their contribution to the model's predictions.

3. **Local Interpretation**:
   - **Force Plots**: Create visualizations for individual predictions to understand why the model made a specific decision.
   - **Dependence Plot**: Explore interactions between a specific feature and the target variable while accounting for other features.

4. **Knowledge Discovery**:
   - Extract insights from feature contributions:
     - Identify **key drivers** influencing the target variable.
     - Uncover **non-linear relationships** and **interaction effects** between features.
   - Use the findings to refine hypotheses or guide domain-specific interventions.

5. **Integration with Domain Expertise**:
   - Cross-validate SHAP insights with domain knowledge to ensure interpretability aligns with real-world phenomena.
   - Use insights for feature engineering or decision-making processes.

By leveraging SHAP, the Random Forest model becomes transparent, enabling both interpretability and actionable insights for improved decision-making.

In [ ]:
import matplotlib
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import plotly.express as px
from shapash.data.data_loader import data_loading
import shap
shap.initjs()   
from shapash.explainer.smart_explainer import SmartExplainer


plt.rcParams ['font.sans-serif'] = ['Times new roman']    
matplotlib.rcParams['axes.unicode_minus'] = False  

import warnings
warnings.filterwarnings('ignore', category=UserWarning)

RF_model = joblib.load("./df3_RandomForest.pkl")

X = df3[df3.columns[:-1]]
y = df3[df3.columns[-1]]

y_pre = pd.DataFrame({'y-pre': RF_model.predict(X)}, index=X.index)
explainer = shap.TreeExplainer(RF_model)
shap_values = explainer.shap_values(X)

# Summary Plot (beeswarm)
shap.summary_plot(shap_values, X, plot_type="dot")
plt.show()

# Summary Plot (bar)
shap.summary_plot(shap_values, X, plot_type="bar")

plt.show()

# Dependence Plot
for feature in X.columns:
    shap.dependence_plot(feature, shap_values, X)
    plt.show()

# Interaction Plot
for feature in X.columns:
    shap.dependence_plot(feature, shap_values, X, interaction_index='auto')
    plt.show()

# heatmap
shap.summary_plot(shap_values, X, plot_type="heatmap")
plt.show()

# Waterfall Plot for the first instance
shap.waterfall_plot(shap.Explanation(values=shap_values[0], base_values=explainer.expected_value, data=X.iloc[0]))
plt.show()

# Force Plot for the first instance
shap.force_plot(explainer.expected_value, shap_values[0], X.iloc[0])
shap.save_html('./shap_force.html', shap.force_plot(explainer.expected_value, shap_values[0], X.iloc[0]))

# Decision Plot
shap.decision_plot(explainer.expected_value, shap_values, X)
plt.show()
